# **Polars en Python: Trabajando Agrupaciones Múltiples, Pivotar y Transponer**

> Notebook para practicar **agrupaciones múltiples**, **pivotar** y **transponer**, usando **Polars**.


## Diferencias clave entre **Pandas** y **Polars** (para alumnos)

- **Motor**: Pandas es muy flexible y muy usado; **Polars** está pensado para ser **muy rápido** y **eficiente en memoria**.
- **Paralelismo**: Polars aprovecha **varios núcleos** (multihilo) en muchas operaciones por defecto.
- **Modo de trabajo**:
  - **Pandas** suele ser *eager* (ejecuta inmediatamente).
  - **Polars** puede ser *eager* **o** *lazy* (planifica un “query plan” y optimiza antes de ejecutar).
- **Expresiones**: en Polars se trabaja mucho con `pl.col(...)`, `pl.when(...).then(...).otherwise(...)`, etc. Eso permite operaciones vectorizadas y optimizables.
- **Índices**:
  - Pandas usa `index` y puede tener **MultiIndex** (muy útil, pero a veces confuso).
  - Polars no centra su diseño en índices; normalmente trabajas con **columnas** (más directo para análisis tipo SQL).
- **Pivot / reshape**: ambos pueden pivotar y “despivotar”, pero la API cambia (en Polars: `pivot`, `unpivot/melt`).

> Consejo: si vienes de Pandas, piensa en Polars como un enfoque “tipo SQL”: **seleccionar columnas, filtrar, agrupar, agregar**.

### **Importar librería**

En Polars, lo habitual es importar como `pl`.

In [1]:
import polars as pl

### **Carga de Datos**

📌 **Idea**: igual que en Pandas, leemos un CSV y revisamos las primeras filas.

> Si estás en Google Colab y tu archivo está en Drive, asegúrate de montar Drive y poner la ruta correcta.

# **AGRUPACIONES MULTIPLES**

En Polars hacemos agrupaciones con `group_by(...).agg(...)`.

- `pl.len()` cuenta filas (equivalente a un `count`).
- Con `alias()` ponemos un nombre más amigable a la columna resultante.

### Agrupación simple

Contamos cuántos registros (por ejemplo, tweets) hay por **mes**.

In [2]:
agrupado = (
    df
    .group_by("mes")
    .agg(pl.len().alias("nº tweets"))
    .sort("mes")
)

agrupado

NameError: name 'df' is not defined

### Agrupar por múltiples niveles en mis dataframes

Ahora contamos por **mes** y **ciudad** (dos columnas a la vez).

In [ ]:
agrupado = (
    df
    .group_by(["mes", "ciudad"])
    .agg(pl.len().alias("nº tweets"))
    .sort(["mes", "ciudad"])
)

agrupado

#### “Reseteo de índice” (comparación con Pandas)

En **Pandas**, una agrupación con varias claves suele devolver un **MultiIndex** y a veces necesitas `reset_index()` para volver a tener columnas.

En **Polars**, el resultado del `group_by` ya viene como **DataFrame con columnas** (`mes`, `ciudad`, `nº tweets`), así que normalmente **no hace falta** “resetear índice”.

In [ ]:
# En Polars no hay MultiIndex como en Pandas.
# El resultado ya trae 'mes' y 'ciudad' como columnas:
desagrupado = agrupado

# Comprobamos columnas disponibles
desagrupado.columns

# **PIVOTAR**

Pivotar es “convertir valores de una columna en columnas”.

Queremos que:
- filas = `mes`
- columnas = `ciudad`
- valores = `nº tweets`

In [ ]:
pivote = (
    desagrupado
    .pivot(
        values="nº tweets",
        index="mes",
        columns="ciudad",
        aggregate_function="first"  # ya está agregado, así que 'first' nos sirve
    )
    .sort("mes")
)

pivote

#### Extraer los datos de una columna

✅ En Polars, como el pivot (con un solo `values`) deja columnas “planas”, podemos seleccionar una ciudad directamente.

> En Pandas a veces aparece MultiIndex en columnas, y por eso se usa `droplevel(...)`. Aquí no hace falta.

In [ ]:
# Opción 1: quedarnos con una sola columna (por ejemplo, GUAYAQUIL)
# (si tu dataset usa otra forma de escribir la ciudad, ajusta el nombre)
pivote.select(["mes", "GUAYAQUIL"])

#### Rellenar valores faltantes

Tras pivotar, puede haber combinaciones `mes-ciudad` que no existan. Eso genera **nulls**.

Los rellenamos con 0 (equivalente a `fillna(0)` en Pandas).

In [ ]:
pivote = pivote.fill_null(0)
pivote

# **TRANSPONER**

Transponer intercambia filas ↔ columnas.

⚠️ Ojo: al transponer, el resultado puede ser menos “cómodo” para análisis; se usa más para presentar o inspeccionar.

In [ ]:
pivote.transpose()